In [10]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(cores=2, memory='20GB', scheduler_address='localhost:8787')
client = Client(cluster)
client

/home/ar1515/.local/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://192.168.8.13:46534 Dashboard: http://192.168.8.13:46747/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [21]:
cluster.scheduler.close()

<coroutine object Scheduler.close at 0x7f6e4a55d248>

In [9]:
# client.cancel()
cluster.close()
client.close()

In [11]:
cluster.scale(10)
client

Client Scheduler: tcp://192.168.8.13:46534 Dashboard: http://192.168.8.13:46747/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
import dask.dataframe as dd
from dask import delayed
import pandas as pd

quotes = dd.read_csv('archive/taq_aug_2019_quotes_500_tickers.csv', dtype={'SYM_SUFFIX': 'object'}, parse_dates=[[0, 1]],
                    infer_datetime_format=True)

# quotes['DATE_TIME'] = quotes.DATE.astype(str) + ' ' + quotes.TIME_M.astype(str)
# quotes.DATE_TIME = dd.to_datetime(quotes.DATE_TIME)
# quotes.drop(['DATE', 'TIME_M', 'SYM_SUFFIX'], axis=1)
quotes.drop('SYM_SUFFIX', axis=1)

# quotes['ALL_BIDS'] = quotes.BID * quotes.BIDSIZ
# quotes['ALL_ASKS'] = quotes.ASK * quotes.ASKSIZ

,DATE_TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,SYM_ROOT
npartitions=3722,,,,,,,
,datetime64[ns],object,float64,int64,float64,int64,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [13]:
trades = dd.read_csv('archive/taq_aug_2019_trades_500_tickers.csv')

trades['DATE_TIME'] = trades.DATE.astype(str) + ' ' + trades.TIME_M.astype(str)
trades.DATE_TIME = dd.to_datetime(trades.DATE_TIME)
trades.drop(['DATE', 'TIME_M', 'SYM_SUFFIX'], axis=1)
# trades.set_index('DATE_TIME')

,EX,SYM_ROOT,SIZE,PRICE,TR_ID,DATE_TIME
npartitions=284,,,,,,
,object,object,int64,float64,int64,datetime64[ns]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [26]:
from datetime import timedelta

start_times = quotes.DATE_TIME - timedelta(minutes=5)
end_times = start_times = timedelta(minutes=180)

In [31]:
quotes_few = quotes.head(10**6)
trades_few = trades.head(10**6)

In [52]:
quotes[quotes.SYM_ROOT == 'A'].set_index('DATE_TIME')

KeyboardInterrupt: 

In [4]:
for sym in quotes.SYM_ROOT.unique():
    quotes[quotes.SYM_ROOT == sym].set_index('DATE_TIME').to_csv('taq_aug_2019_quotes_dask'+sym+'.csv', single_file=True)

KilledWorker: ("('assign-from-delayed-pandas_read_text-read-block-shuffle-join-shuffle-group-97d61fd756ef959c29a187434a19a02c', 1, (13, 15, 11))", <Worker 'tcp://192.168.8.44:43481', memory: 0, processing: 137>)

In [ ]:
import numpy as np
x = client.submit(np.unique, client.scatter(quotes.SYM_ROOT))
x.result()